# 地理信息系统设计与开发（第二次课）

In [10]:
import ogr
import osr
import json
import requests
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
url="http://127.0.0.1/all_month.json"

In [12]:
try:
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    r.encoding = r.apparent_encoding
    print(requests.head(url).headers)
    print(r.text[0:1000])
 
except:
    print("error!")

{'Content-Length': '10041290', 'Content-Type': 'application/json', 'Last-Modified': 'Tue, 12 May 2020 12:17:10 GMT', 'Accept-Ranges': 'bytes', 'ETag': '"74ea42435728d61:0"', 'Server': 'Microsoft-IIS/10.0', 'Date': 'Thu, 11 Jun 2020 07:04:02 GMT'}
{"type":"FeatureCollection","metadata":{"generated":1589285573000,"url":"https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.geojson","title":"USGS All Earthquakes, Past Month","status":200,"api":"1.8.1","count":14011},"features":[{"type":"Feature","properties":{"mag":2.27,"place":"28km SE of Bodie, CA","time":1589285365660,"updated":1589285464488,"tz":-480,"url":"https://earthquake.usgs.gov/earthquakes/eventpage/nc73383965","detail":"https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/nc73383965.geojson","felt":null,"cdi":null,"mmi":null,"alert":null,"status":"automatic","tsunami":0,"sig":79,"net":"nc","code":"73383965","ids":",nc73383965,","sources":",nc,","types":",geoserve,nearby-cities,origin,phase-data,","nst":18,"dm

# 通过ogr的GetDriverByName方法创建driver

In [16]:
driver=ogr.GetDriverByName('ESRI Shapefile')

## 通过driver的CreateDataSource方法创建datasource

In [19]:
ds=driver.CreateDataSource('test3.shp')

### 通过datasource的CreateLayer方法创建layer


In [20]:
spatial_reference=osr.SpatialReference()
spatial_reference.ImportFromEPSG(4326)
layer=ds.CreateLayer('myEarthQuake',spatial_reference,geom_type=ogr.wkbPoint)

#### 通过layer的CreateField方法创建属性表结构


In [23]:
layer.CreateField(ogr.FieldDefn('magnitude',ogr.OFTReal))
layer.CreateField(ogr.FieldDefn('place',ogr.OFTString))
layer.CreateField(ogr.FieldDefn('time',ogr.OFTReal))
layer.CreateField(ogr.FieldDefn('id',ogr.OFTString))
layer.CreateField(ogr.FieldDefn('longitude',ogr.OFTReal))
layer.CreateField(ogr.FieldDefn('latitude',ogr.OFTReal))
layer.CreateField(ogr.FieldDefn('depth',ogr.OFTReal))               

0

##### 通过json获取相关数据
##### 通过layer的GetlayerDefn方法获取feature的属性表结构
##### 通过ogr创建新的feature
##### 设定feature中的相关字段值
##### 通过ogr的Geometry类创建point对象
##### 通过point的AddPoint方法设定坐标位置
##### 通过feature的SetGeometry方法将feature设定为新建的point
##### 通过layer的CreateFeature方法将point feature添加到layer

In [26]:
geojson=json.loads(r.text)
features=geojson['features']
for feat in features:
    #get properties
    properties=feat['properties']
    magnitude=properties['mag']
    place=properties['place']
    time=properties['time']
    id=properties['id']
    geom=feat['geometry']
    longitude=geom['coordinates'][0]
    latitude=geom['coordinates'][1]
    depth=geom['coordinates'][2]
    #print(depth)
    #set field
    feature_defn=layer.GetLayerDefn()
    feature=ogr.Feature(feature_defn)
    feature.SetField('magnitude',magnitude)
    feature.SetField('place',place)
    feature.SetField('time',time)
    feature.SetField('longitude',longitude)
    feature.SetField('latitude',latitude)
    feature.SetField('depth',depth)
    #create point
    point=ogr.Geometry(ogr.wkbPoint)
    point.AddPoint(longitude,latitude)
    feature.SetGeometry(point)
    
    layer.CreateFeature(feature)
    
    plt.plot(longitude,latitude,'o',markersize=5,color='k') 
    

In [28]:
del ds